In [8]:
#Выполните сохранение монохромного изображения в виде текстового или бинарного файла.
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

img_color = cv.imread('sar_1_gray.jpg')
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
np.savetxt('sar_1_gray.txt', img_gray, fmt='%d')

print("Монохромное изображение сохранено в текстовый файл sar_1_gray.txt")


Монохромное изображение сохранено в текстовый файл sar_1_gray.txt


In [6]:
#Реализуйте алгоритм вейвлет-преобразования Хаара для изображения.
import cv2 as cv
import numpy as np

img_color = cv.imread('sar_1_gray.jpg')
img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)

img = np.float32(img_gray)

rows, cols = img.shape
rows_even = rows if rows % 2 == 0 else rows - 1
cols_even = cols if cols % 2 == 0 else cols - 1
img = img[0:rows_even, 0:cols_even]

h = 1.0 / np.sqrt(2.0)


def haar_1d(signal):
    n = signal.shape[0]
    result = np.zeros_like(signal)
    half = n // 2
    for i in range(0, n, 2):
        a = (signal[i] + signal[i + 1]) * h
        d = (signal[i] - signal[i + 1]) * h
        result[i // 2] = a
        result[half + i // 2] = d
    return result

temp = np.zeros_like(img)
for r in range(rows_even):
    temp[r, :] = haar_1d(img[r, :])

haar_img = np.zeros_like(temp)
for c in range(cols_even):
    haar_img[:, c] = haar_1d(temp[:, c])

print("Вейвлет-преобразование Хаара выполнено.")


Вейвлет-преобразование Хаара выполнено.


In [13]:
#Выполните квантование высокочастотных компонент (прим., количество квантов = 4).
import numpy as np
import cv2 as cv

levels = 4 

def quantize_band(band, levels=4):
    band_min = band.min()
    band_max = band.max()
    if band_max == band_min:
        return band.copy()
    step = (band_max - band_min) / levels

    q_idx = np.floor((band - band_min) / step)
    q_idx = np.clip(q_idx, 0, levels - 1)

    band_q = band_min + (q_idx + 0.5) * step
    return band_q

haar_q = haar_img.copy()

rows, cols = haar_q.shape
half_r, half_c = rows // 2, cols // 2

HL = haar_q[0:half_r, half_c:cols]
LH = haar_q[half_r:rows, 0:half_c]
HH = haar_q[half_r:rows, half_c:cols]

haar_q[0:half_r, half_c:cols]      = quantize_band(HL, levels)
haar_q[half_r:rows, 0:half_c]      = quantize_band(LH, levels)
haar_q[half_r:rows, half_c:cols]   = quantize_band(HH, levels)

print("Квантование выполнено.")


Квантование выполнено.


In [14]:
#Сохраните получившийся массив значений в текстовый или бинарный файл в порядке LL,
#LH, HL, HH вейвлет-преобразования Хафа. Компоненты LH, HL, HH храните в виде пар
#(значение, количество повторений).
import numpy as np

rows, cols = haar_q.shape
half_r, half_c = rows // 2, cols // 2

LL = haar_q[0:half_r, 0:half_c].flatten()
LH = haar_q[half_r:rows, 0:half_c].flatten()
HL = haar_q[0:half_r, half_c:cols].flatten()
HH = haar_q[half_r:rows, half_c:cols].flatten()

def rle_encode(arr):
    encoded = []
    if len(arr) == 0:
        return encoded
    prev = arr[0]
    count = 1
    for x in arr[1:]:
        if x == prev:
            count += 1
        else:
            encoded.append((prev, count))
            prev = x
            count = 1
    encoded.append((prev, count))
    return encoded

LH_rle = rle_encode(LH)
HL_rle = rle_encode(HL)
HH_rle = rle_encode(HH)

with open("haar_output.txt", "w") as f:
    f.write("LL:\n")
    f.write(" ".join(map(str, LL)))
    f.write("\n\nLH (value,count):\n")
    for v, c in LH_rle:
        f.write(f"{v},{c} ")
    f.write("\n\nHL (value,count):\n")
    for v, c in HL_rle:
        f.write(f"{v},{c} ")
    f.write("\n\nHH (value,count):\n")
    for v, c in HH_rle:
        f.write(f"{v},{c} ")

print("Сохранение выполнено: файл haar_output.txt создан.")


Сохранение выполнено: файл haar_output.txt создан.


In [16]:
#Сравните объем памяти, занимаемый исходным изображением (попиксельное хранение), и
#изображение, полученным после преобразования Хафа и сжатием длин серий.
import numpy as np

rows, cols = img_gray.shape
original_pixels = rows * cols

r2, c2 = haar_q.shape
half_r, half_c = r2 // 2, c2 // 2

LL = haar_q[0:half_r, 0:half_c].flatten()
LH = haar_q[half_r:r2, 0:half_c].flatten()
HL = haar_q[0:half_r, half_c:c2].flatten()
HH = haar_q[half_r:r2, half_c:c2].flatten()

def rle_encode(arr):
    encoded = []
    if len(arr) == 0:
        return encoded
    prev = arr[0]
    count = 1
    for x in arr[1:]:
        if x == prev:
            count += 1
        else:
            encoded.append((prev, count))
            prev = x
            count = 1
    encoded.append((prev, count))
    return encoded

LH_rle = rle_encode(LH)
HL_rle = rle_encode(HL)
HH_rle = rle_encode(HH)

original_values = original_pixels
LL_values = LL.size

LH_values = 2 * len(LH_rle)
HL_values = 2 * len(HL_rle)
HH_values = 2 * len(HH_rle)

compressed_values = LL_values + LH_values + HL_values + HH_values

compression_ratio = compressed_values / original_values

print(f"Размер исходного изображения (в значениях): {original_values}")
print(f"Размер после Хаара + RLE (в значениях):   {compressed_values}")
print(f'Коэффициент сжатия (после / до):          {compression_ratio:.3f}')
print(f'Сжатие в раз:                              {1/compression_ratio:.3f}')


Размер исходного изображения (в значениях): 240000
Размер после Хаара + RLE (в значениях):   172012
Коэффициент сжатия (после / до):          0.717
Сжатие в раз:                              1.395
